# Es 1.. 2.1

In [2]:
import networkx as nx
import pandas as pd
import json
import ijson
import time
import csv
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import ast
from itertools import combinations

In [3]:
citation_graph = nx.DiGraph()
collaboration_graph = nx.Graph()
collaboration_graph_prova = nx.Graph()

In [4]:
df_sorted =pd.read_csv('C:/Users/Gabriele/Desktop/università/Magistrale/DATA SCIENCE/ADM/HW5/dataset_sorted_top10000.tsv',sep='\t')

In [5]:
df_sorted.tail()

,id,title,year,authors,n_citation,doc_type,reference_count,references,doi,publisher
9995,1964830323,An overview of JML tools and applications,2005.0,"[{'name': 'Lilian Burdy', 'org': 'INRIA Sophia...",596.0,Conference,82.0,1486696980;1489778371;1492315860;1498946538;14...,https://doi.org/10.1007/s10009-004-0167-4,Springer-Verlag
9996,2101699859,Countering code-injection attacks with instruc...,2003.0,"[{'name': 'Gaurav S. Kc', 'org': 'Columbia Uni...",596.0,Conference,42.0,186343359;1481758559;1499992849;1508969946;151...,https://doi.org/10.1145/948109.948146,ACM
9997,2124609748,Gaussian Process Dynamical Models for Human Mo...,2008.0,"[{'name': 'J.M. Wang', 'org': 'University of T...",596.0,Conference,61.0,1505866674;1546296670;1598702468;1643263348;19...,https://doi.org/10.1109/TPAMI.2007.1167,IEEE Computer Society
9998,2147343704,EVENODD: an efficient scheme for tolerating do...,1995.0,"[{'name': 'M. Blaum', 'org': 'IBM Almaden Rese...",596.0,Journal,9.0,1530042190;1531975040;1820898047;1829547464;20...,https://doi.org/10.1109/12.364531,IEEE Computer Society
9999,2093212899,Simultaneous structure and texture image inpai...,2003.0,"[{'name': 'M. Bertalmio', 'org': 'Dept. de Tec...",596.0,Journal,22.0,1488881187;1565233179;1569587969;1983661653;19...,https://doi.org/10.1109/TIP.2003.815261,IEEE


In [6]:
def fill_nodes_citation(data, G):
    for _, row in data.iterrows():
        paper_id = row['id']
        references_str = row['references']

        # Convert the references string to a list of integers
        references = [int(ref) for ref in str(references_str).split(';') if ref.isdigit()]

        # Add edges to the graph if the reference exists in the dataset
        for ref in references:
            if ref in data['id'].values:
                G.add_edge(paper_id, ref)


In [7]:
fill_nodes_citation(df_sorted, citation_graph)

In [8]:
def fill_nodes_collaboration(data, G):
    for _, row in data.iterrows():
        authors_str = row['authors']
        title = row['title']  

        try:
            authors = ast.literal_eval(authors_str)
            # Extract a unique identifier for each author (e.g., name or ID)
            author_ids = [author.get('name') or author.get('id') for author in authors]
        except ValueError:
            author_ids = []

        author_pairs = list(combinations(author_ids, 2))

        for author_pair in author_pairs:
            if G.has_edge(*author_pair):
                G[author_pair[0]][author_pair[1]]['weight'] = G[author_pair[0]][author_pair[1]]['weight'] + 1 if 'weight' in G[author_pair[0]][author_pair[1]] else 1
            else:
                G.add_edge(author_pair[0], author_pair[1], weight=1, titles=title)


In [9]:
fill_nodes_collaboration(df_sorted, collaboration_graph)

# 2.1

## Functionality 1

In [10]:
def analyze_graph(graph, graph_name):
    # Number of nodes
    num_nodes = graph.number_of_nodes()

    # Number of edges
    num_edges = graph.number_of_edges()

    # Graph density
    density = nx.density(graph)

    # Degree distribution
    degree_sequence = [d for n, d in graph.degree()]
    degree_distribution = dict(zip(*np.unique(degree_sequence, return_counts=True)))

    # Average degree
    avg_degree = np.mean(degree_sequence)

    # 95th percentile of the degree distribution
    percentile_95 = np.percentile(degree_sequence, 95)

    # Graph hubs (nodes with degrees > 95th percentile)
    hubs = [node for node, degree in graph.degree() if degree > percentile_95]

    # Whether the graph is dense or sparse
    graph_type = "dense" if density >= 0.5 else "sparse"

    # Report
    print(f"Analysis Report for Graph: {graph_name}")
    print("--------------------------------------------------")
    print(f"Number of Nodes: {num_nodes}")
    print(f"Number of Edges: {num_edges}")
    print(f"Graph Density: {density}")
    print(f"Graph Degree Distribution: {degree_distribution}")
    print(f"Average Degree: {avg_degree}")
    print(f"Graph Hubs: {hubs}")
    print(f"Graph Type: {graph_type}\n")



In [14]:
analyze_graph(citation_graph, "Citation Graph")
analyze_graph(collaboration_graph, "Collaboration Graph")


Analysis Report for Graph: Citation Graph
--------------------------------------------------
Number of Nodes: 9429
Number of Edges: 52289
Graph Density: 0.0005882001442304685
Graph Degree Distribution: {1: 597, 2: 681, 3: 639, 4: 590, 5: 585, 6: 607, 7: 585, 8: 521, 9: 476, 10: 419, 11: 405, 12: 363, 13: 345, 14: 259, 15: 240, 16: 209, 17: 224, 18: 181, 19: 162, 20: 145, 21: 138, 22: 105, 23: 113, 24: 87, 25: 65, 26: 70, 27: 63, 28: 48, 29: 47, 30: 38, 31: 39, 32: 25, 33: 25, 34: 17, 35: 32, 36: 18, 37: 19, 38: 16, 39: 15, 40: 14, 41: 12, 42: 13, 43: 18, 44: 5, 45: 4, 46: 5, 47: 9, 48: 12, 49: 7, 50: 6, 51: 5, 52: 3, 53: 4, 54: 7, 55: 6, 56: 7, 57: 5, 58: 2, 59: 5, 60: 5, 61: 5, 62: 1, 63: 3, 64: 4, 65: 5, 66: 4, 67: 2, 68: 2, 69: 2, 70: 5, 71: 1, 73: 2, 74: 2, 75: 1, 76: 2, 77: 1, 79: 1, 80: 2, 81: 1, 86: 1, 89: 2, 90: 1, 91: 2, 92: 1, 93: 2, 95: 1, 102: 1, 104: 1, 120: 1, 122: 1, 144: 1, 152: 1}
Average Degree: 11.091101919609715
Graph Hubs: [2151103935, 1676552347, 2024668293, 21098

## Functionality 2

In [13]:
import networkx as nx

def calculate_centrality(graph, node, graph_name):
    """
    Calculate centrality measurements for a given node in the graph.

    Parameters:
    - graph: The graph (NetworkX graph object).
    - node: The node for which centrality is to be calculated.
    - graph_name: The name of the graph.

    Returns:
    - centrality_measures: A dictionary containing centrality measurements (Betweenness, PageRank, ClosenessCentrality, DegreeCentrality).
    """
    centrality_measures = {}

    # Betweenness Centrality
    betweenness_centrality = nx.betweenness_centrality(graph)
    centrality_measures['Betweenness'] = betweenness_centrality.get(node, 0)

    # PageRank
    pagerank_centrality = nx.pagerank(graph)
    centrality_measures['PageRank'] = pagerank_centrality.get(node, 0)

    # Closeness Centrality
    closeness_centrality = nx.closeness_centrality(graph)
    centrality_measures['ClosenessCentrality'] = closeness_centrality.get(node, 0)

    # Degree Centrality
    degree_centrality = nx.degree_centrality(graph)
    centrality_measures['DegreeCentrality'] = degree_centrality.get(node, 0)

    print(f"Centrality Measures for {node} in {graph_name}:")
    for measure, value in centrality_measures.items():
        print(f"{measure}: {value}")

    return centrality_measures


In [14]:
calculate_centrality(collaboration_graph, 'E Dimmer', "Collaboration Graph")


Centrality Measures for E Dimmer in Collaboration Graph:
Betweenness: 0.0
PageRank: 2.7300433210642505e-05
ClosenessCentrality: 0.02800701455614984
DegreeCentrality: 0.006050129645635264


{'Betweenness': 0.0,
 'PageRank': 2.7300433210642505e-05,
 'ClosenessCentrality': 0.02800701455614984,
 'DegreeCentrality': 0.006050129645635264}

## Functionality 3

In [15]:
from collections import deque

for author in collaboration_graph.nodes:
    collaboration_graph.nodes[author]['papers_published'] = len(list(collaboration_graph.neighbors(author)))

#Functionality 3 function
def combined_shortest_path(graph, start, end, intermediate_nodes, N):
    combined_path = {'path': [], 'titles': []}

    # Helper function to get top N nodes based on papers published
    def get_top_nodes(graph, N):
        return sorted(graph.nodes, key=lambda x: graph.nodes[x]['papers_published'], reverse=True)[:N]

    # Helper function to perform BFS
    def BFS(graph, start, end, top_nodes):
        path_queue = deque()

        # Each element in the queue is a tuple (current_path, titles_list)
        temp_path = ([start], [])
        path_queue.append(temp_path)

        while path_queue:
            # Dequeue the next path and its associated titles
            current_path, current_titles = path_queue.popleft()
            last_node = current_path[-1]

            # Check if the last node in the path is the target
            if last_node == end:
                return current_path, current_titles

            # Explore neighbors of the last node
            for link_node, edge_data in graph[last_node].items():
                # Check if the neighbor is not already in the current path and is in the top N nodes
                if link_node not in current_path and link_node in top_nodes:
                    # Extend the path and titles with the new neighbor and its associated titles
                    new_path = current_path + [link_node]
                    new_titles = current_titles + [edge_data.get('titles', [])]
                    path_queue.append((new_path, new_titles))

        # If no valid path is found
        return [], []


    # Get the top N nodes based on papers published
    top_nodes = get_top_nodes(graph, N)

    # Calculate the first path and titles
    first_path, first_titles = BFS(graph, start, intermediate_nodes[0], top_nodes)
    combined_path['path'] += first_path
    combined_path['titles'] += first_titles

    if not first_path:
        return "There is no such path."

    start = intermediate_nodes[0]

    # Iterate through intermediate nodes and calculate the shortest path for each pair
    for intermediate_node in intermediate_nodes[1:]:
        path, titles = BFS(graph, start, intermediate_node, top_nodes)
        combined_path['path'] += path[1:]
        combined_path['titles'] += titles

        if not path:
            return "There is no such path."

        start = intermediate_node

    # Calculate the shortest path from the last intermediate node to the end
    last_path, last_titles = BFS(graph, intermediate_nodes[-1], end, top_nodes)
    combined_path['path'] += last_path[1:]
    combined_path['titles'] += last_titles

    if not last_path:
        return "There is no such path."

    return combined_path

# Example usage
first_node = 'E Dimmer'
last_node = 'C Bonilla'
intermediate_nodes = ['WM Chan']
N = 1000 # Replace with your desired N value

# Adding 'papers_published' attribute to nodes in the collaboration graph (replace this with your actual data)
for author in collaboration_graph.nodes:
    collaboration_graph.nodes[author]['papers_published'] = len(list(collaboration_graph.neighbors(author)))

result = combined_shortest_path(collaboration_graph, first_node, last_node, intermediate_nodes, N)

if isinstance(result, dict):
    print("Shortest Path: ", result['path'])
    print("Titles: ", result['titles'])
else:
    print(result)


Combined Shortest Path:  ['E Dimmer', 'WM Chan', 'C Bonilla']
Combined Titles:  ['The Universal Protein Resource (UniProt) in 2010', 'Activities at the Universal Protein Resource (UniProt)']
